# Experiment LLM using API



### API key

In [ ]:
import openai

# set API key
client = openai.OpenAI(api_key="")  # fill in API key

### Prompt categories

In [ ]:
# Zero-shot category
def prompt_zero_shot(puzzle):
    return f"""Los het volgende taalkundige raadsel op. Het bevat drie aanwijzingen en het antwoord is één woord dat op alle drie van toepassing is. Geef het antwoord op de eerste regel en daarna een korte uitleg.

Vraag: {puzzle}. Wat is het?
Antwoord:"""

# One-shot category
def prompt_one_shot(puzzle):
    return f"""Los het volgende taalkundige raadsel op. Het bevat drie aanwijzingen en het antwoord is één woord dat op alle drie van toepassing is. Geef het antwoord op de eerste regel en daarna een korte uitleg. Eerst een voorbeeld, daarna een nieuw raadsel.

Voorbeeld:
Vraag: Het is een onderdeel van een schip, een bevestigingsmiddel, en een gymnastiekoefening. Wat is het?
Antwoord: Schroef.

Nu het raadsel:
Vraag: {puzzle}. Wat is het?
Antwoord:"""

# Three-shot category
def prompt_three_shot(puzzle):
    return f"""Los het volgende taalkundige raadsel op. Het bevat drie aanwijzingen en het antwoord is één woord dat op alle drie van toepassing is. Geef het antwoord op de eerste regel en daarna een korte uitleg. Eerst drie voorbeelden, dan een nieuw raadsel.

Voorbeeld 1:
Vraag: Het is een onderdeel van een schip, een bevestigingsmiddel, en een gymnastiekoefening. Wat is het?
Antwoord: Schroef.

Voorbeeld 2:
Vraag: Het is een winkelketen, een beroep, en een onderdeel van een schip. Wat is het?
Antwoord: Zeeman.

Voorbeeld 3:
Vraag: Het is seksuele anatomie, een beledigend woord, en een vrucht. Wat is het?
Antwoord: Eikel.

Raadsel:
Vraag: {puzzle}. Wat is het?
Antwoord:"""

### Define Files

In [ ]:
# File used for experiment
infile = "test_puzzles.txt" # set infile

# File used to save results
outfile = "results_test.txt" # set outfile

### Run API

In [ ]:
import ast


# Read file
def getData(file):
    with open(file, "r", encoding="utf-8") as f:
        return f.readlines()

# Write file
def writeData(file, data):
    with open(file, "w", encoding="utf-8") as f:
        for line in data:
            f.write(f"{line}\n")

# Load data from infile
data = getData(infile)

results = []
correct = 0
for i, page in enumerate(data, 1):
    load = ast.literal_eval(page)
    puzzle = load["prompt"]
    answer = load["answer"].strip().lower()

    # Set prompt category
    prompt = prompt_three_shot(puzzle) # set shot category

    print(f"Processing: {i+1}/{len(data)}")

    try:
        # Generate output
        response = client.chat.completions.create(
            model="o4-mini", # set model
            messages=[
                {"role": "system", "content": "Je bent een taalexpert die raadsels oplost."},
                {"role": "user", "content": prompt}
            ]
        )
        result = response.choices[0].message.content.strip()

        # Evaluate result
        if answer in predicted.strip().lower():
            correct += 1

        results.append({"prompt": puzzle, "answer": answer, "result": result})

    except Exception as e:
        print(f"Error at prompt {i}: {e}")
        results.append({
            "prompt": puzzle,
            "expected": answer,
            "predicted": "ERROR",
            "full_response": str(e)
        })

print(f"Correct: {correct}/{len(data)}")

# Export results
writeData(outfile, [str(r) for r in results])